# Advanced spatial joins

## Lecture objectives

1. Learn about spatial predicates
2. Learn about distance and nearest neighbor calculations
3. Gain practice with troubleshooting spatial joins

**You'll need to add the pygeos library in order to do the nearest neighbors**. I omitted this when we set up the environment at the start of the quarter.

Install it as follows from the command line:

`conda activate uds`

`conda install pygeos`

You can also open up Anaconda in the point-and-click. Make sure it goes into your `uds` environment.

## Other types of spatial relationships

The `intersects` operator, which we just used, is one of the most common. It is excellent if you want to know whether a point is within (or on the boundary) of a polygon. In general, you'll get a 1:1 match, but you need to be careful if your points are on the boundary and thus intersect with two polygons.

Let's start by loading two of the datasets we used in the last lecture. This is exactly the same code.

In [ ]:
import requests
import json
import geopandas as gpd
import pandas as pd

# get food pantry dataset via Socrata
r  = requests.get('https://controllerdata.lacity.org/resource/uztv-ve9b.json')
pantryDf = pd.DataFrame(json.loads(r.content))

# convert to a GeoDataFrame
pantrygdf = gpd.GeoDataFrame(
    pantryDf, geometry=gpd.points_from_xy(pantryDf.longitude, pantryDf.latitude, 
                                          crs='EPSG:4326'))
# convert to 3857 to match the census data
pantrygdf.to_crs('EPSG:3857', inplace=True)

# get the census data for the City of LA
# B19019_001E is median household income
from cenpy import products
incomeDf = products.ACS(2017).from_place('Los Angeles, CA', level='tract',
                                        variables='B19019_001E')
incomeDf.rename(columns={'B19019_001E':'median_HH_income'}, inplace=True)

These predicates are defined in the `shapely` [documentation](https://shapely.readthedocs.io/en/stable/manual.html#binary-predicates).

In [ ]:
print(incomeDf.sindex.valid_query_predicates)

Let's look at the number of rows that are produced with three different predicates: intersects, contains, and within.

How do you explain the different results below?

In [ ]:
print('There are {} pantries and {} tracts.'.format(len(pantrygdf), len(incomeDf)))

pantries_intersects = gpd.sjoin(incomeDf[['GEOID','geometry']], 
                           pantrygdf, how="inner", predicate='intersects')
print('Using intersects: {} rows'.format(len(pantries_intersects)))

pantries_contains = gpd.sjoin(incomeDf[['GEOID','geometry']], 
                           pantrygdf, how="inner", predicate='contains')
print('Using contains: {} rows'.format(len(pantries_contains)))

pantries_within = gpd.sjoin(incomeDf[['GEOID','geometry']], 
                           pantrygdf, how="inner", predicate='within')
print('Using within: {} rows'.format(len(pantries_within)))


Intersects and contains return the same number of rows. If a census tract (a polygon) intersects a pantry (a point geometry), it also contains it.

But a census tract is not within a pantry. 

## Troubleshooting spatial joins

Also note that we have fewer join results than pantries. Why? A good guess would be that some pantries are outside the City of Los Angeles.

<div class="alert alert-block alert-info">
<strong>Thought exercise:</strong> How would you verify that the missing pantries are outside the city?
</div>

We could map them. But note that the pantries dataset has a city field. So let's do `groupby` and ask for the size of each group.

In [ ]:
pantrygdf.groupby('city').size()

Hmm. So either all the pantries are within the city, or there is an error in the `city` column. Or our join is going wrong.

Let's map it and take a look. Note that we use the `alpha` parameter to adjust the transparency of both the census tracts and the basemap, making it easier to see the intersection.

In [ ]:
import matplotlib.pyplot as plt
import contextily as ctx

fig, ax = plt.subplots(figsize=(10,10))
pantrygdf.plot(ax=ax)
incomeDf.plot(ax=ax, alpha=0.5) # 50% opacity
ctx.add_basemap(ax=ax, zoom=12, alpha=0.3) # 30% opacity

Aha. If you recognize the shape of the city of Los Angeles, it looks like `cenpy` is not giving us all the tracts.

Let's troubleshoot. Our first stop will be the [documentation](https://cenpy-devs.github.io/cenpy/generated/cenpy.products.ACS.from_place.html).

There are a few things we could try:
* Provide the `place_type` argument as the docs recommend
* Experiment with the `strict_within` argument. Perhaps we are losing the tracts on the edge of the city boundary if this is `True` by default?
* Try a different census variable

In [ ]:
print(len(incomeDf))
incomeDf = products.ACS(2017).from_place('Los Angeles, CA', 
                                          place_type='Incorporated Place',
                                          level='tract', variables='B19019_001E')
print(len(incomeDf))

No, that didn't do it. We still get 749 tracts.

What about the `strict_within` argument?

In [ ]:
incomeDf = products.ACS(2017).from_place('Los Angeles, CA', 
                                          place_type='Incorporated Place',
                                          level='tract', strict_within=False,
                                          variables='B19019_001E')
incomeDf.rename(columns={'B19019_001E':'median_HH_income'}, inplace=True)
print(len(incomeDf))

Aha. Fixed it. Now let's run our join again.

In [ ]:
pantries_intersects = gpd.sjoin(incomeDf[['GEOID','geometry']], 
                           pantrygdf, how="inner", predicate='intersects')
print(len(pantries_intersects))

Now we are just missing one! Let's map it again to take a look. This is identical code to before.

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
pantrygdf.plot(ax=ax)
incomeDf.plot(ax=ax, alpha=0.5) # 50% opacity
ctx.add_basemap(ax=ax, zoom=12, alpha=0.3) # 30% opacity

It looks like two of them are just on the boundary. Let's adjust the axis limits to see if that's the case. That will zoom on the affected area.

We can set the limits by eyeballing from the figure above.

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))
pantrygdf.plot(ax=ax)
incomeDf.plot(ax=ax, alpha=0.5) # 50% opacity
ctx.add_basemap(ax=ax, zoom=12, alpha=0.3) # 30% opacity

ax.set_xlim([-1.316e7, -1.315e7])
ax.set_ylim([4.03e6, 4.04e6])

It looks like the geometry is imprecise, or perhaps the pantry is just outside of the city limits. Perhaps we could correct the coordinates manually, or ask `cenpy` for all tracts in LA County (not just the city).

The point here: spatial joins often throw up unexpected results. You'll have to do some detective work to figure out what goes wrong.

## Spatial attributes
We can access useful attributes of the geometry directly from `geopandas`. Areas, length, and bounding boxes are three examples.

Note that the units will be based on your projection. You probably don't want to measure distances or areas in degrees! But even a projection in meters can be distorted.

The best projection will vary locally. In the US, the State Plane coordinate systems are usually the best choice for local-level work. Los Angeles is in State Plane [California zone 5, which has an EPSG code of 3497](https://epsg.io/3497). Note the units are in meters. If you want feet, [try EPSG code 6424](https://epsg.io/6424). 

In [ ]:
# in Web Mercator
print(incomeDf.geometry.area.head())

In [ ]:
# in State Plane
print(incomeDf.to_crs('EPSG:3497').geometry.area.head())

For a polygon, length is the perimeter.

In [ ]:
print(incomeDf.geometry.to_crs('EPSG:3497').length.head()) 

The projection is less important for the bounding box, as long as you know which one you are using. Here, we are still in Web Mercator. But lat/lon is often the easiest to work with.

In [ ]:
print(incomeDf.geometry.bounds.head())

In [ ]:
print(incomeDf.to_crs('EPSG:4326').geometry.bounds.head())

## Distances and nearest neighbors
Another common use case is getting the distances between a geometry and a set of other geometries, or the nearest neighbor. Again, the projection is important here so let's convert to State Plane. 

In [ ]:
pantrygdf.to_crs('EPSG:3497', inplace=True)
incomeDf.to_crs('EPSG:3497', inplace=True)

The nearest neighbor can be found with `sjoin_nearest`. The optional argument, `distance_col`, will add a column with the distances.

In [ ]:
incomeDf.sjoin_nearest(pantrygdf, distance_col='dist_to_pantry')

Note that we only have the result for the closest part of the census tract. If we want the centroid, we can create a new GeoDataFrame and convert its polygons to centroids.

In [ ]:
incomeDf_centroids = incomeDf.copy()
incomeDf_centroids.geometry = incomeDf.geometry.centroid

# map to show the centroids 
fig, ax=plt.subplots(figsize=(10,10))
incomeDf_centroids.plot(ax=ax)
incomeDf.plot(ax=ax, lw=4, alpha=0.5)

And let's do the nearest neighbor with these centroids.

In [ ]:
incomeDf_centroids.sjoin_nearest(pantrygdf, distance_col='dist_to_pantry')

Notice that the distances are a little larger than before.

In [ ]:
incomeDf.sjoin_nearest(pantrygdf, 
        distance_col='dist_to_pantry').dist_to_pantry.mean()

In [ ]:
incomeDf_centroids.sjoin_nearest(pantrygdf, 
        distance_col='dist_to_pantry').dist_to_pantry.mean()

What if you don't just care about the closest one, but want to get the distances from a census tract to a larger number of pantries, or even all of them? For example, some accessibility measures look at the distance to the 2nd or 3rd closest destination (e.g. a grocery store), in order to capture the number of choices that people have.

To start with, let's look at the distances to a single tract. Note that `sort_values` will sort the results, so it's easiest to see the smallest and largest distances.

In [ ]:
# take the first census tract, and get its geometry
tract = incomeDf.iloc[0].geometry

# get the distances from this tract to all the food pantries
distances = pantrygdf.distance(tract)
distances.sort_values(inplace=True)
distances

So how do we know which one is the 3rd closest? We can use `iloc` to get the 3rd row. 

In [ ]:
distances.iloc[2]

If we want to calculate the distance to the 3rd closest pantry for each census tract, we can put this in a function.

The argument of the function will be the geometry of the tract. It will return the distance.

Once we have that function, we can use our old friend `apply` to apply it to every tract in the city of LA.

In [ ]:
def get_3rd_closest_dist(geom):
    distances = pantrygdf.distance(geom)
    third_closest = distances.sort_values().iloc[2]
    return third_closest

incomeDf['dist_third_closest'] = incomeDf.geometry.apply(get_3rd_closest_dist)

In [ ]:
incomeDf

Finally, let's plot using the `seaborn.regplot()` function that we saw before.

In [ ]:
import seaborn as sns
ax = sns.regplot(x="median_HH_income", y="dist_third_closest", data=incomeDf)

<div class="alert alert-block alert-info">
<h3>Key Takeaways</h3>
<ul>
  <li>Intersects is the most common spatial predicate, but it's not always what you want.</li>
  <li>Areas, lengths, nearest neighbors, and distances are simple to calculate in geopandas.</li>
  <li>Use an an appropriate projection.</li>
  <li>As with any join, inspect your output. If it's not what you expect, computing summary statistics or mapping can help identify the problem.</li>
</ul>
</div>